In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import DenseNet201, ResNet152V2, VGG16
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Concatenate, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import os
import zipfile

In [ ]:
# Ensure the dataset is extracted from Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Define paths to dataset
zip_path = '/content/drive/MyDrive/Dataset/SkinDiseases.zip'
extract_path = '/content/skin_disease_dataset'

# Extract the dataset
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print('Dataset extracted successfully.')

Dataset extracted successfully.


In [ ]:
data_dir_train = os.path.join(extract_path, 'SkinDisease/train')
data_dir_test = os.path.join(extract_path, 'SkinDisease/test')

In [ ]:
# Data Preprocessing and Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Split for training and validation
)


In [ ]:
train_generator = train_datagen.flow_from_directory(
    data_dir_train,
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)


Found 11128 images belonging to 22 classes.


In [ ]:
validation_generator = train_datagen.flow_from_directory(
    data_dir_train,
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 2770 images belonging to 22 classes.


In [ ]:
# Fusion Model Development
input_shape = (256, 256, 3)
input_tensor = Input(shape=input_shape, name="input_image")

In [ ]:
# DenseNet201 Model
input_tensor_densenet = Input(shape=input_shape)
base_model_densenet = DenseNet201(weights='imagenet', include_top=False, input_tensor=input_tensor_densenet)
base_model_densenet.trainable = False  # Freeze layers
x_densenet = base_model_densenet.output
x_densenet = GlobalAveragePooling2D()(x_densenet)
x_densenet = Dense(512, activation='relu')(x_densenet)
output_densenet = Dense(train_generator.num_classes, activation='softmax')(x_densenet)

model_densenet = Model(inputs=input_tensor_densenet, outputs=output_densenet)

# Compile the model
model_densenet.compile(optimizer=Adam(),
                       loss='categorical_crossentropy',
                       metrics=['accuracy'])
# Callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.75, patience=5, min_lr=1e-5, verbose=1)
# early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)

# Train the model
history_densenet = model_densenet.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=30,
    callbacks=[reduce_lr]
)

Epoch 1/30


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


348/348 ━━━━━━━━━━━━━━━━━━━━ 367s 920ms/step - accuracy: 0.3292 - loss: 2.3012 - val_accuracy: 0.3513 - val_loss: 2.1501 - learning_rate: 0.0010
Epoch 2/30
348/348 ━━━━━━━━━━━━━━━━━━━━ 293s 830ms/step - accuracy: 0.4892 - loss: 1.6516 - val_accuracy: 0.3708 - val_loss: 2.1420 - learning_rate: 0.0010
Epoch 3/30
348/348 ━━━━━━━━━━━━━━━━━━━━ 297s 763ms/step - accuracy: 0.5451 - loss: 1.4670 - val_accuracy: 0.3606 - val_loss: 2.2386 - learning_rate: 0.0010
Epoch 4/30
348/348 ━━━━━━━━━━━━━━━━━━━━ 266s 755ms/step - accuracy: 0.5832 - loss: 1.3405 - val_accuracy: 0.3787 - val_loss: 2.1764 - learning_rate: 0.0010
Epoch 5/30
348/348 ━━━━━━━━━━━━━━━━━━━━ 266s 755ms/step - accuracy: 0.6193 - loss: 1.2223 - val_accuracy: 0.3866 - val_loss: 2.2048 - learning_rate: 0.0010
Epoch 6/30
348/348 ━━━━━━━━━━━━━━━━━━━━ 269s 764ms/step - accuracy: 0.6309 - loss: 1.1690 - val_accuracy: 0.3866 - val_loss: 2.2572 - learning_rate: 0.0010
Epoch 7/30
348/348 ━━━━━━━━━━━━━━━━━━━━ 0s 594ms/step - accuracy: 0.6612 - 

In [ ]:
model_densenet.save('/content/drive/MyDrive/my_model.h5')